# Money Laudering Detection Through Social Network Analysis

This notebook discusses the sophisticated layering strategies used by criminals to cover money laundering schemes and possible ways to detect the association between the sender and the receiver. The problem statement can be summarized as follows,

Money laundering (ML) is the process of transforming the profits of crime and corruption into legitimate assets. To this end, criminal organizations and entities use the legal banking system to ingest money acquired from illegal activities, which is later withdrawn as legitimate payments.

One ML technique used by criminals is the so-called "bridging", where an entity A sends money to an entity B via an entity C (i.e., A->C->B), creating a layer of dissociation between entities A and B. In one instance, a drug dealer 'John Smith' may want to pay his immediate drug distributor 'Mary Jerry'. They may then agree to use Mary's brother-in-law 'Joseph Louis' to intermediate the transaction, serving as a "bridge" between John and Mary. Joseph therefore will start forwarding every payment made by John to Mary and, depending on the agreement, may take a percentage of the payments as fees for his "services". Purpose of the current notebook is to scan financial records in search of bridging transactions and entities suspected of bridging and finally sort the transactions based on the suspicion level from high to low.

The data for this exercise has an unique identifier, a timestamp, the transferred amount, the sender's account id and the receiver's account id. Given that the data does not have labels, I analyzed the transactions as a graph network and investigate the red flags between transactions. My approach can be summarized as follows,
* I grouped the transactions based on a timeframe. (The timeframe is a variable but for this discussion it is set to 1 day.)
* For each given timeframe, look at the total money transfer from each account. If the transfer from an account is below a cut off value, I assumed that the money transfer is 'not worth' tracking. (Again this cut off value is a variable and based on the sensitivity of the analysis it can be tuned. I set it to $10000, if one wants to be more conservative, this value can be decreased.)
* For the remaining accounts, I constructed a graph network where the entities are the vertices and the transferred money is the edges connecting these vertices. Just to be clear, each day has its own network.
* Looking at these networks, I applied another layer of filter based on the number of transfers happening in between accounts. If the money transfer chain has fewer than two edges within the given timeframe, get rid of these transactions.
* To separate the 'clean' transaction chains from the possibly money laundering groups, I applied one last layer of filtering. If at any point in the transfer chain, the ratio of the money received by an account to the money leaving the initial sender is above a threshold value, I kept these groups as highly suspicious. (The threshold value is a variable and it is assumed to be 0.9 for this discussion.)
* Finally, I sorted the highly suspicious accounts and transfers by measuring certain features of these subgraphs, such as; degree centrality, closeness centrality and betweenness centrality. 

This filtering approach significantly reduces the number of the transactions to a manageable subset. An example of a money laundering network is shown in the figure below. Although this graph by no means represents all money laundering schemes, I wanted to motivate my model by showing a few important points on this graph. Here, node A sends money to node Z via nodes B, C, E and D. The first important point is that the transfer is divided into chunks and sent through different bridging nodes. These transfers are modelled by directed edges between nodes. Additionally, a node does not have to send the money as a one time transfer. The total amount can be divided into fractions and can be sent different times of the day, as shown by the edges in between nodes E and Z.

![alt text](graph.png "Title")

The details of the filtering analysis is as follows. For cleanliness, I kept the helper functions out of the discussion, they can be reached as a separate file.

## Filtering Before the Graph Construction

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
import networkx as nx
from datetime import datetime

Before constructing the graphs, initial filtering was suitable for distributed computing so I took advantage of Spark.

In [ ]:
money_cut_off = 10000.
commission_cut_off = 0.9
group_by_time = 1 #days

sc = SparkContext(appName = "graph")
lines = sc.textFile('./transactions.csv')
header = lines.first()
lines = lines.filter(lambda x: x != header)
lines = lines.filter(lambda x: x.split('|')[1] != '2006-02-29')

I prescribed the cut off variables and timeframe as described above. The data did not require almost any cleaning or imputation. The only problem was that there were transactions recorded on 02-29-2006, which is not an actual date, so I had to remove those transactions.

In [ ]:
filtered_by_amount_per_node = get_amount_per_node_transfer(lines)
filtered_transactions = get_filtered_transactions(lines, filtered_by_amount_per_node)        
nodes_edge_weights_dict, nodes_edge_weights_lst = get_nodes_and_edges(filtered_transactions)

First, I grouped the transactions by the date and the sender account to understand the total sent money from each account per given timeframe. The timeframe can be tuned by increments of a day as I converted the timestamp into the proleptic Gregorian date. This conversion returns an integer which is the day count from 01/01/01 until the given date. Therefore, transactions can be grouped by an integer with floor division. Then, I distinguished and removed the transactions involving accounts that transfer below $\$$10000 for a given day. I did not do this money filtering on each transaction because as described previously some accounts might divide the cut off value ($\$$10000) into multiple chunks and send as batches. The total amount of money transfer per day per node is stored in a dictionary after the filtering.

## The Graph Analysis

In [ ]:
normalized_transactions, betweenness_dict, transaction_volume, closeness_dict = [], {}, {}, {}

for day_transactions in nodes_edge_weights_lst:
    #Creates different graphs for different days
    G = create_graph([el[1:] for el in day_transactions])
    max_subgraph_size, visited_subgraphs = 0, []
    
    for transaction in day_transactions:
        # The depth of the transaction chain starting from the inital sender
        depth = get_depth(G, transaction[1])
        if depth > 1:            
            #get connected nodes in bfs order
            connected_nodes = get_connected_nodes(G, transaction[1])

            if max_subgraph_size  < len(connected_nodes) or not set(connected_nodes) <= set(visited_subgraphs):
                max_subgraph_size = len(connected_nodes)
                visited_subgraphs += connected_nodes
                
                # I looked at the closeness and betweenness for only 
                # transaction groups with multiple edges.
                # This saved significant amount of computational time 
                # as graph size decreases once the 
                # transactions with single edges are filtered.
                sub_G = G.subgraph(connected_nodes)
                betweenness_nodes = nx.betweenness_centrality(sub_G, normalized=True)
                closeness_nodes = nx.closeness_centrality(sub_G)
                
                # Edge weights are normalized by the money initially 
                # sent from the sender.
                # Also in and out degree weights are calculated for 
                # each node.
                normalized_transaction, transaction_volume = get_normalized_edge_weights(
                                                    G, transaction[0], connected_nodes, transaction_volume)
                
                if normalized_transaction not in normalized_transactions:         
                    normalized_transactions.append(normalized_transaction)
                                    

The graph analysis involves the following methodology,
* Create a graph for each day. Meaning, each day is separately analyzed. Correlations over time is not considered.
* Starting from a node, get all the connected nodes with bread first search order. Depth first search order would also work but I found bread first search more intuitive for this problem as money flows layer by layer away from the inital account.
* Construct subgraphs by using these subgroups of connected nodes to calculate betweenness and closeness. This filtering before the betweenness and closeness calculations significantly cut the computational time.
* Similar to the closeness and betweenness, a parameter is described which measures the frequency of money flow over the nodes (I will call it transaction traffic parameter for the rest of the notebook). It is defined as the total number of inbound and outbound transfers per node on a given day normalized by the maximum number of inbound and outbound transfers on that day which maps the parameter into [0,1] range.
* Normalize each edge weight aka each money transfer by the total money initially sent from the first node.

In [ ]:
#fix multiple edge receiver problem
suspicious_transactions = get_suspicious_transactions(normalized_transactions, commission_cut_off)
       
#normalize the transaction traffic parameter
suspicious_traffic = get_suspicious_traffic(transaction_volume) 

Some further manipulation is needed in case multiple edges are pointing to the same receiver node. The multiple transactions from a source to the same receiver node strategy might be used by the criminals to confuse financial crime investigators. Each transaction might seem small but when the total transferred money is consider on a given day, it might be comparable to the initally sent money. If an edge in the money transfer chain has a weight above a threshold value (in this case I set it to 0.9) and below 1, that means 90-100% of the initially sent money is received by a connected node. The 10% gap in the ratio is set according to the problem description. These group of transactions are separated as highly suspicious transaction chains which are to be further manipulated.

## Risk Assesment

The transactions and the entities are ranked based on how suspicious they are according to the following methodology,

* A gamma parameter is calculated per node as a linear combination of the node's traffic, betweenness and closeness parameters. There was not any labeled data to tune the weights of these parameters so I referred mostly to [2] while deciding on the weights. A labelled data set would be helpful in the case of a real application to cross validate and change these weights.
* The highest contribution to the gamma parameteer comes from the traffic paramter, then betweenness parameter and a small contribution from the closeness parameter.
* The gamma parameter describes the 'suspicion' of a given node. The higher the gamma parameter, the higher the risk that the given node is involved in a money laundering scheme at a given day. An important detail here is that, although I analyzed each day on its own during the graph analysis, for the final gamma parameter calculation, I looked at the total value of the gamma values accumulated over time. Therefore, a node which has a very high gamma parameter for one day might be less suspicious than another node which has smaller gamma parameters over a long timeframe. This takes into account the historical activity of each node.
* For each given transaction chain, the entities involved in the money transfer, determines the total 'suspicion' of that transfer chain. Thus, graphs involving highly risky entities has the highest suspicion.

A small note here is that; I did not look at the individual transactions, rather I considered a full chain of transactions when I analyze the suspicion level of the transactions. Therefore, each chain has the same gamma parameter and consequently the same suspicion level.

In [ ]:
#gamma parameters calculated for each node and sorted from high to low
gamma_nodes, gamma_dict = get_gamma(suspicious_traffic, betweenness_dict, closeness_dict)

#gamma parameters calculated for each transaction chain
suspicious_transactions_by_line = get_suspicious_transactions(suspicious_transactions, gamma_nodes)                            

I detected more than 40000 entities involved in possible money laundering transactions which makes in total more than half a million transactions. It is almost 10% of the total transaction number in the data set and flagging all these transactions as money laundering would probably end up with a massive number of false positives. To prevent this, it would be nice to have a set of labelled data where I can understand the correlation of gamma parameter with money laundering activity. This would be helpful to set a threshold gamma where the transactions above it would be flagged as money laundering and vice versa. As it is not the case, I reported all the possible entities and transactions involved in the chain activities from high to low risk.

One last point is that, although I tried to model the problem as general as possible, the money laundering schemes can be highly sophisticated. Learning from the data might be superior to the rule-based approaches. An unsupervised graph learning model such as; DeepWalk might be useful to learn the social relations of the nodes in a graph model. These relations can be further analyzed to detect anomalies. However, given the timeframe and the computational constraints, I decided to not pursue this way when I attempted this specific problem.

For further reference, this analysis is highly inspired by these two papers:

[1] Soltani, Reza, et al. "A new algorithm for money laundering detection based on structural similarity." 2016 IEEE 7th Annual Ubiquitous Computing, Electronics & Mobile Communication Conference.

[2] Colladon, Andrea Fronzetti, and Elisa Remondi. "Using social network analysis to prevent money laundering." 2017 Expert Systems with Applications.